Setup Environment

In [8]:
# Clear the R environment and set working directory
rm(list = ls())
setwd("/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters")
path <- getwd()
path 

[1] "/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters"

Load packages

In [9]:
# Install and load additional packages needed
#install.packages("did")
library(did)
library(rio)
library(ggplot2)
library(ggpubr)
#library(cowplot)
library(hash)
library(patchwork)
.libPaths()

[1] "/Library/Frameworks/R.framework/Versions/4.2-arm64/Resources/library"

DiD for BTW no controls

In [10]:
# read in the data set
btw <- import(sprintf("%s/data/btw_treat.csv", path))
# Filter for first and second vote
erst <- subset(btw, first_vote == 1)
zweit <- subset(btw, second_vote == 1)

In [11]:
parties <- c('Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere')
treatments <- hash('treatment_0'='Direct Line', 'treatment_15'='Within 15km', 'treatment_30'='Within 30km', 'treatment_50'='Within 50km')
keys <- keys(treatments)
results  <- list()
for(party in parties){
  figures  <- list()
  for(treatment in keys){
    result <- att_gt(yname = party,
                  gname = treatment,
                  idname = 'AGS',
                  panel = TRUE,
                  tname = 'year',
                  xformla = ~ 1,
                  data = erst,
                  est_method = 'reg',
                  anticipation = 0,
                  control_group = 'nevertreated',
                  clustervars = c('AGS'),
                  bstrap = TRUE,
                  cband = TRUE,
                  allow_unbalanced_panel = TRUE,
                  
    )
    results[[length(results)+1]] <- result
    # calculate ATT
    att <- aggte(result, type = "group", bstrap = TRUE, clustervars = c('AGS'))
    # plot results
    result_fig <- ggdid(result, 
              ylim = c(floor(min(result$att - result$se * 2.345 - 1)), ceiling(max(result$att + result$se * 2.345 + 1))),
              ncol = 3, ax_text_size = 8, grtitle='', title_size = 10, legend=F) +
              labs(caption = sprintf('Overall ATT (SE): %.3f (%.3f) \n Wald-P: %.3f \n # Obs.: %s' ,
              att$overall.att, att$overall.se, result$Wpval, result$n)) +
              theme(plot.caption = element_text(hjust=0.5, size=8),
              panel.background = element_rect(fill='transparent'),
              plot.background = element_rect(fill='transparent', color=NA),
              legend.background = element_rect(fill='transparent'),
              legend.box.background = element_rect(fill='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
    # Plot ATT
    att_fig <- ggdid(att, legend=F, x_lab='', ax_text_size = 8, ylab='', title='ATT by year', title_size = 10) + 
    labs(caption = sprintf('2005 ATT(SE): %.3f (%.3f)\n2010 ATT(SE): %.3f (%.3f)\n2014 ATT(SE): %.3f (%.3f)', 
              att$att.egt[[1]], att$se.egt[[1]], att$att.egt[[2]], att$se.egt[[2]], att$att.egt[[3]], att$se.egt[[3]])) + 
              theme(plot.caption = element_text(hjust=0.5, size=8), 
              axis.title.x = element_blank(),
              axis.text.y = element_text(angle=90, size=6, hjust=0.5),
              panel.background = element_rect(fill='transparent'),
              plot.background = element_rect(fill='transparent', color=NA),
              legend.background = element_rect(fill='transparent'),
              legend.box.background = element_rect(fill='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
    # Combine and annotate Group and ATT plot
    combined_fig <- ggarrange(result_fig, att_fig, widths = c(3, 1)) + 
                    theme(plot.margin=unit(c(0,0,0,0),"cm"))
    combined_fig_anno <- annotate_figure(combined_fig,
                  top = text_grob(sprintf('%s', treatments[[treatment]])))
    figures[[length(figures)+1]] <- combined_fig_anno
  }
  # Combine Figures for all treatments
  arranged_fig <- ggarrange(plotlist=figures, nrow = 4, ncol = 1, common.legend = TRUE)
  # get legend
  result_fig <- ggdid(result) +
              theme(
              legend.background = element_rect(fill='transparent', color='transparent'),
              legend.box.background = element_rect(fill='transparent', color='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
  leg <- get_legend(result_fig)
  # add legend
  arranged_fig_leg <- ggarrange(arranged_fig, leg, nrow = 2, ncol = 1, heights = c(10, 1))
  # Annotate final figure
  final_fig <- annotate_figure(arranged_fig_leg,
                  top = text_grob(sprintf("BTW: Effect on %s's vote share", party), face = "bold", size = 14),
                  bottom = text_grob('Control Group: Never Treated. Estimation Method: Regression \n SE clustered on the municipality level.', size = 10))
  ggsave(sprintf('%s.png', party), plot = final_fig, path = sprintf('%s/figures/R/BTW/pooled', path), units = 'cm', width = 21, height = 21, dpi="print")
  print(party)
}

[1] "Union"
[1] "SPD"
[1] "FDP"
[1] "Linke"
[1] "Grüne"
[1] "Andere"


DiD for BTW with controls

In [12]:
btw_c <- import(sprintf("%s/data/btw_control.csv", path))
erst_c <- subset(btw_c, first_vote == 1)

In [13]:
parties <- c('Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere')
treatments <- hash('treatment_0'='Direct Line', 'treatment_15'='Within 15km', 'treatment_30'='Within 30km', 'treatment_50'='Within 50km')
keys <- keys(treatments)
results  <- list()
for(party in parties){
  figures  <- list()
  for(treatment in keys){
    result <- att_gt(yname = party,
                  gname = treatment,
                  idname = 'AGS',
                  panel = TRUE,
                  tname = 'year',
                  xformla = ~ east + pop_density + unemployed + female + avg_age,
                  data = erst_c,
                  est_method = 'reg',
                  anticipation = 0,
                  control_group = 'nevertreated',
                  clustervars = c('AGS'),
                  bstrap = TRUE,
                  cband = TRUE,
                  allow_unbalanced_panel = TRUE,
                  
    )
    results[[length(results)+1]] <- result
    # calculate ATT
    att <- aggte(result, type = "group", bstrap = TRUE, clustervars = c('AGS'))
    # plot results
    result_fig <- ggdid(result, 
              ylim = c(floor(min(result$att - result$se * 2.345 - 1)), ceiling(max(result$att + result$se * 2.345 + 1))),
              ncol = 3, ax_text_size = 8, grtitle='', title_size = 10, legend=F) +
              labs(caption = sprintf('Overall ATT (SE): %.3f (%.3f) \n Wald-P: %.3f \n # Obs.: %s' ,
              att$overall.att, att$overall.se, result$Wpval, result$n)) +
              theme(plot.caption = element_text(hjust=0.5, size=8),
              panel.background = element_rect(fill='transparent'),
              plot.background = element_rect(fill='transparent', color=NA),
              legend.background = element_rect(fill='transparent'),
              legend.box.background = element_rect(fill='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
    # Plot ATT
    att_fig <- ggdid(att, legend=F, x_lab='', ax_text_size = 8, ylab='', title='ATT by year', title_size = 10) + 
    labs(caption = sprintf('2005 ATT(SE): %.3f (%.3f)\n2010 ATT(SE): %.3f (%.3f)\n2014 ATT(SE): %.3f (%.3f)', 
              att$att.egt[[1]], att$se.egt[[1]], att$att.egt[[2]], att$se.egt[[2]], att$att.egt[[3]], att$se.egt[[3]])) + 
              theme(plot.caption = element_text(hjust=0.5, size=8), 
              axis.title.x = element_blank(),
              axis.text.y = element_text(angle=90, size=6, hjust=0.5),
              panel.background = element_rect(fill='transparent'),
              plot.background = element_rect(fill='transparent', color=NA),
              legend.background = element_rect(fill='transparent'),
              legend.box.background = element_rect(fill='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
    # Combine and annotate Group and ATT plot
    combined_fig <- ggarrange(result_fig, att_fig, widths = c(3, 1)) + 
                    theme(plot.margin=unit(c(0,0,0,0),"cm"))
    combined_fig_anno <- annotate_figure(combined_fig,
                  top = text_grob(sprintf('%s', treatments[[treatment]])))
    figures[[length(figures)+1]] <- combined_fig_anno
  }
  # Combine Figures for all treatments
  arranged_fig <- ggarrange(plotlist=figures, nrow = 4, ncol = 1, common.legend = TRUE)
  # get legend
  result_fig <- ggdid(result) +
              theme(
              legend.background = element_rect(fill='transparent', color='transparent'),
              legend.box.background = element_rect(fill='transparent', color='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
  leg <- get_legend(result_fig)
  # add legend
  arranged_fig_leg <- ggarrange(arranged_fig, leg, nrow = 2, ncol = 1, heights = c(10, 1))
  # Annotate final figure
  final_fig <- annotate_figure(arranged_fig_leg,
                  top = text_grob(sprintf("BTW: Effect on %s's vote share", party), face = "bold", size = 14),
                  bottom = text_grob('Control Group: Never Treated. Estimation Method: Regression \n SE clustered on the municipality level.', size = 10))
  ggsave(sprintf('%s.png', party), plot = final_fig, path = sprintf('%s/figures/R/BTW/control', path), units = 'cm', width = 21, height = 21, dpi="print")
  print(party)
}

Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”


[1] "Union"


Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”


[1] "SPD"


Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”


...

[1] "Linke"


Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”


[1] "Grüne"


Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 2 rows from original data due to missing data”


[1] "Andere"


DiD for BTW with controls by state

In [14]:
btw_c <- import(sprintf("%s/data/btw_control.csv", path))
erst_c <- subset(btw_c, first_vote == 1)

In [19]:
parties <- c('Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere')
treatments_dict <- hash('treatment_0'='Direct Line', 'treatment_15'='Within 15km', 'treatment_30'='Within 30km', 'treatment_50'='Within 50km')
states_dict <- hash('SH'='Schleswig-Holstein (SH)', 'NI'='Lower Saxony (NI)', 'NW'='North-Rhine Westphalia (NRW)', 'HE'='Hesse (HE)', 
'RP'='Rhineland-Palatine (RP)', 'BW'='Baden-Württemberg (BW)', 'BY'='Bavaria (BY)', 'BB'='Brandenburg (BB)', 'MV'='Mecklenburg-Vorpommern (MV)', 
'ST'='Saxony-Anhalt (ST)', 'TH'='Thuringia (TH)') # all states - those never treated
treatments <- keys(treatments_dict)
states <- keys(states_dict)
results  <- list()
for (state in states){
  for(party in parties){
    reg_data <- subset(erst_c, Land == state)
    figures  <- list()
    for(treatment in treatments){
      result <- att_gt(yname = party,
                    gname = treatment,
                    idname = 'AGS',
                    panel = TRUE,
                    tname = 'year',
                    xformla = ~ pop_density + unemployed + female,
                    data = reg_data,
                    est_method = 'reg',
                    anticipation = 0,
                    control_group = 'nevertreated',
                    clustervars = c('AGS'),
                    bstrap = TRUE,
                    cband = TRUE,
                    allow_unbalanced_panel = TRUE,
                    
      )
    results[[length(results)+1]] <- result
    # calculate ATT
    att <- aggte(result, type = "group", bstrap = TRUE, clustervars = c('AGS'))
    # plot results
    result_fig <- ggdid(result, 
              ylim = c(floor(min(result$att - result$se * 2.345 - 1)), ceiling(max(result$att + result$se * 2.345 + 1))),
              ncol = 3, ax_text_size = 8, grtitle='', title_size = 10, legend=F) +
              labs(caption = sprintf('Overall ATT (SE): %.3f (%.3f) \n Wald-P: %.3f \n # Obs.: %s' ,
              att$overall.att, att$overall.se, result$Wpval, result$n)) +
              theme(plot.caption = element_text(hjust=0.5, size=8),
              panel.background = element_rect(fill='transparent'),
              plot.background = element_rect(fill='transparent', color=NA),
              legend.background = element_rect(fill='transparent'),
              legend.box.background = element_rect(fill='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
    # Plot ATT
    att_fig <- ggdid(att, legend=F, x_lab='', ax_text_size = 8, ylab='', title='ATT by year', title_size = 10) + 
    try(
      labs(caption = sprintf('2005 ATT(SE): %.3f (%.3f)\n2010 ATT(SE): %.3f (%.3f)\n2014 ATT(SE): %.3f (%.3f)', 
              att$att.egt[[1]], att$se.egt[[1]], att$att.egt[[2]], att$se.egt[[2]], att$att.egt[[3]], att$se.egt[[3]]))
    ) + 
    try(
      labs(caption = sprintf('2010 ATT(SE): %.3f (%.3f)\n2014 ATT(SE): %.3f (%.3f)', 
              att$att.egt[[1]], att$se.egt[[1]], att$att.egt[[2]], att$se.egt[[2]]))
    ) +
              theme(plot.caption = element_text(hjust=0.5, size=8), 
              axis.title.x = element_blank(),
              axis.text.y = element_text(angle=90, size=6, hjust=0.5),
              panel.background = element_rect(fill='transparent'),
              plot.background = element_rect(fill='transparent', color=NA),
              legend.background = element_rect(fill='transparent'),
              legend.box.background = element_rect(fill='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
    # Combine and annotate Group and ATT plot
    combined_fig <- ggarrange(result_fig, att_fig, widths = c(3, 1)) + 
                    theme(plot.margin=unit(c(0,0,0,0),"cm"))
    combined_fig_anno <- annotate_figure(combined_fig,
                  top = text_grob(sprintf('%s', treatments[[treatment]])))
    figures[[length(figures)+1]] <- combined_fig_anno
    }
    # Combine Figures for all treatments
    arranged_fig <- ggarrange(plotlist=figures, nrow = 4, ncol = 1, common.legend = TRUE)
    # get legend
    result_fig <- ggdid(result) +
                theme(
                legend.background = element_rect(fill='transparent', color='transparent'),
                legend.box.background = element_rect(fill='transparent', color='transparent'),
                plot.margin=unit(c(0,0,0,0),"cm"))
    leg <- get_legend(result_fig)
    # add legend
    arranged_fig_leg <- ggarrange(arranged_fig, leg, nrow = 2, ncol = 1, heights = c(10, 1))
    # Annotate final figure
    final_fig <- annotate_figure(arranged_fig_leg,
                    top = text_grob(sprintf("BTW in %s\nEffect on %s's vote share", states_dict[[state]], party), face = "bold", size = 14),
                    bottom = text_grob('Control Group: Never Treated. Estimation Method: Regression \n SE clustered on the municipality level.', size = 10),)
    ggsave(sprintf('%s_%s.png', state, party), plot = final_fig, path = sprintf('%s/figures/R/BTW/by_state', path), units = 'cm', width = 21, height = 29.7, dpi="print")
    print(state)
    print(party)
  }
}

Warning message in compute.aggte(MP = MP, type = type, balance_e = balance_e, min_e = min_e, :
“Simultaneous conf. band is somehow smaller than pointwise one using normal approximation. Since this is unusual, we are reporting pointwise confidence intervals”


Error in att$att.egt[[3]] : subscript out of bounds


ERROR: [1m[33mError[39m in [1m[1m`ggplot_add()`:[22m
[33m![39m Can't add `try(labs(caption = sprintf("2005 ATT(SE): %.3f (%.3f)\n2010 ATT(SE): %.3f (%.3f)\n2014 ATT(SE): %.3f (%.3f)", ` to a ggplot object.
[36m•[39m Can't add `    att$att.egt[[1]], att$se.egt[[1]], att$att.egt[[2]], att$se.egt[[2]], ` to a ggplot object.
[36m•[39m Can't add `    att$att.egt[[3]], att$se.egt[[3]])))` to a ggplot object.


DiD for LTW no controls

In [20]:
# read in the data set
ltw <- import(sprintf("%s/data/ltw_treat.csv", path))

In [21]:
parties <- c('Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere')
treatments <- hash('treatment_0'='Direct Line', 'treatment_15'='Within 15km', 'treatment_30'='Within 30km', 'treatment_50'='Within 50km')
keys <- keys(treatments)
results  <- list()
for(party in parties){
  figures  <- list()
  for(treatment in keys){
    result <- att_gt(yname = party,
                  gname = treatment,
                  idname = 'AGS',
                  panel = TRUE,
                  tname = 'year',
                  xformla = ~ 1,
                  data = ltw,
                  est_method = 'reg',
                  anticipation = 0,
                  control_group = 'nevertreated',
                  clustervars = c('AGS'),
                  bstrap = TRUE,
                  cband = TRUE,
                  allow_unbalanced_panel = TRUE,
                  
    )
  results[[length(results)+1]] <- result
  # calculate ATT
  att <- aggte(result, type = "group", bstrap = TRUE, clustervars = c('AGS'))
  # plot results
  result_fig <- ggdid(result, 
            ylim = c(floor(min(result$att - result$se * 2.345 - 1)), ceiling(max(result$att + result$se * 2.345 + 1))),
            ncol = 3, ax_text_size = 8, grtitle='', title_size = 10, legend=F) +
            labs(caption = sprintf('Overall ATT (SE): %.3f (%.3f) \n Wald-P: %.3f \n # Obs.: %s' ,
            att$overall.att, att$overall.se, result$Wpval, result$n)) +
            theme(plot.caption = element_text(hjust=0.5, size=8),
            panel.background = element_rect(fill='transparent'),
            plot.background = element_rect(fill='transparent', color=NA),
            legend.background = element_rect(fill='transparent'),
            legend.box.background = element_rect(fill='transparent'),
            plot.margin=unit(c(0,0,0,0),"cm"))
  # Plot ATT
  att_fig <- ggdid(att, legend=F, x_lab='', ax_text_size = 8, ylab='', title='ATT by year', title_size = 10) + 
  labs(caption = sprintf('2005 ATT(SE): %.3f (%.3f)\n2010 ATT(SE): %.3f (%.3f)\n2014 ATT(SE): %.3f (%.3f)', 
            att$att.egt[[1]], att$se.egt[[1]], att$att.egt[[2]], att$se.egt[[2]], att$att.egt[[3]], att$se.egt[[3]])) + 
            theme(plot.caption = element_text(hjust=0.5, size=8), 
            axis.title.x = element_blank(),
            axis.text.y = element_text(angle=90, size=6, hjust=0.5),
            panel.background = element_rect(fill='transparent'),
            plot.background = element_rect(fill='transparent', color=NA),
            legend.background = element_rect(fill='transparent'),
            legend.box.background = element_rect(fill='transparent'),
            plot.margin=unit(c(0,0,0,0),"cm"))
  # Combine and annotate Group and ATT plot
  combined_fig <- ggarrange(result_fig, att_fig, widths = c(3, 1)) + 
                  theme(plot.margin=unit(c(0,0,0,0),"cm"))
  combined_fig_anno <- annotate_figure(combined_fig,
                top = text_grob(sprintf('%s', treatments[[treatment]])))
  figures[[length(figures)+1]] <- combined_fig_anno
  }
  # Combine Figures for all treatments
  arranged_fig <- ggarrange(plotlist=figures, nrow = 4, ncol = 1, common.legend = TRUE)
  # get legend
  result_fig <- ggdid(result) +
              theme(
              legend.background = element_rect(fill='transparent', color='transparent'),
              legend.box.background = element_rect(fill='transparent', color='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
  leg <- get_legend(result_fig)
  # add legend
  arranged_fig_leg <- ggarrange(arranged_fig, leg, nrow = 2, ncol = 1, heights = c(10, 1))
  final_fig <- annotate_figure(arranged_fig_leg,
                  top = text_grob(sprintf("LTW: Effect on %s's vote share", party), face = "bold", size = 14),
                  bottom = text_grob('Control Group: Never Treated. Estimation Method: Regression \n SE clustered on the municipality level.', size = 10),)
  ggsave(sprintf('%s_all.png', party), plot = final_fig, path = sprintf('%s/figures/R/LTW/pooled', path), units = 'cm', width = 21, height = 29.7, dpi="print")
  print(party)
}

Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“dropped 1010 rows from original data due to missing data”
Warning message in pre_process_did(yname = yname, tname = tname, idname = idname, :
“Dropped 227 units that were already treated in the first period.”


ERROR: Error in att$att.egt[[3]]: subscript out of bounds


DiD for LTW no controls by state

In [ ]:
parties <- c('Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere')
treatments_dict <- hash('treatment_0'='Direct Line', 'treatment_15'='Within 15km', 'treatment_30'='Within 30km', 'treatment_50'='Within 50km')
states_dict <- hash('NI'='Lower Saxony (NI)', 'NW'='North-Rhine Westphalia (NRW)', 'HE'='Hesse (HE)', 'BY'='Bavaria (BY)')
treatments <- keys(treatments_dict)
states <- keys(states_dict)
results  <- list()
for (state in states){
  for(party in parties){
    figures  <- list()
    reg_data <- subset(ltw, Land == state)
    for(treatment in treatments){
      result <- att_gt(yname = party,
                    gname = treatment,
                    idname = 'AGS',
                    panel = TRUE,
                    tname = 'year',
                    xformla = ~ 1,
                    data = reg_data,
                    est_method = 'reg',
                    anticipation = 0,
                    control_group = 'nevertreated',
                    clustervars = c('AGS'),
                    bstrap = TRUE,
                    cband = TRUE,
                    allow_unbalanced_panel = TRUE,
                    
      )
    results[[length(results)+1]] <- result
    # calculate ATT
    att <- aggte(result, type = "group", bstrap = TRUE, clustervars = c('AGS'))
    # plot results
    result_fig <- ggdid(result, 
              ylim = c(floor(min(result$att - result$se * 2.345 - 1)), ceiling(max(result$att + result$se * 2.345 + 1))),
              ncol = 3, ax_text_size = 8, grtitle='', title_size = 10, legend=F) +
              labs(caption = sprintf('Overall ATT (SE): %.3f (%.3f) \n Wald-P: %.3f \n # Obs.: %s' ,
              att$overall.att, att$overall.se, result$Wpval, result$n)) +
              theme(plot.caption = element_text(hjust=0.5, size=8),
              panel.background = element_rect(fill='transparent'),
              plot.background = element_rect(fill='transparent', color=NA),
              legend.background = element_rect(fill='transparent'),
              legend.box.background = element_rect(fill='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
    # Plot ATT
    att_fig <- ggdid(att, legend=F, x_lab='', ax_text_size = 8, ylab='', title='ATT by year', title_size = 10) + 
    labs(caption = sprintf('2005 ATT(SE): %.3f (%.3f)\n2010 ATT(SE): %.3f (%.3f)\n2014 ATT(SE): %.3f (%.3f)', 
              att$att.egt[[1]], att$se.egt[[1]], att$att.egt[[2]], att$se.egt[[2]], att$att.egt[[3]], att$se.egt[[3]])) + 
              theme(plot.caption = element_text(hjust=0.5, size=8), 
              axis.title.x = element_blank(),
              axis.text.y = element_text(angle=90, size=6, hjust=0.5),
              panel.background = element_rect(fill='transparent'),
              plot.background = element_rect(fill='transparent', color=NA),
              legend.background = element_rect(fill='transparent'),
              legend.box.background = element_rect(fill='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
    # Combine and annotate Group and ATT plot
    combined_fig <- ggarrange(result_fig, att_fig, widths = c(3, 1)) + 
                    theme(plot.margin=unit(c(0,0,0,0),"cm"))
    combined_fig_anno <- annotate_figure(combined_fig,
                  top = text_grob(sprintf('%s', treatments[[treatment]])))
    figures[[length(figures)+1]] <- combined_fig_anno
    }
    # Combine Figures for all treatments
    arranged_fig <- ggarrange(plotlist=figures, nrow = 4, ncol = 1, common.legend = TRUE)
    # get legend
    result_fig <- ggdid(result) +
                theme(
                legend.background = element_rect(fill='transparent', color='transparent'),
                legend.box.background = element_rect(fill='transparent', color='transparent'),
                plot.margin=unit(c(0,0,0,0),"cm"))
    leg <- get_legend(result_fig)
    # add legend
    arranged_fig_leg <- ggarrange(arranged_fig, leg, nrow = 2, ncol = 1, heights = c(10, 1))
    final_fig <- annotate_figure(arranged_fig_leg,
                    top = text_grob(sprintf("LTW %s \nEffect on %s's vote share", state, party), face = "bold", size = 14),
                    bottom = text_grob('Control Group: Never Treated. Estimation Method: Regression \n SE clustered on the municipality level.', size = 10),)
    ggsave(sprintf('%s_%s_nc.png', state, party), plot = final_fig, path = sprintf('%s/figures/R/LTW/by_state', path), units = 'cm', width = 21, height = 29.7, dpi="print")
    print(party)
  }
}

DiD for LTW controls by state

In [ ]:
ltw_c <- import(sprintf("%s/data/ltw_control.csv", path))

In [ ]:
parties <- c('Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere')
treatments_dict <- hash('treatment_0'='Direct Line', 'treatment_15'='Within 15km', 'treatment_30'='Within 30km', 'treatment_50'='Within 50km')
states_dict <- hash('NI'='Lower Saxony (NI)', 'NW'='North-Rhine Westphalia (NRW)', 'HE'='Hesse (HE)', 'BY'='Bavaria (BY)')
treatments <- keys(treatments_dict)
states <- keys(states_dict)
results  <- list()
for (state in states){
  for(party in parties){
    figures  <- list()
    reg_data <- subset(ltw_c, Land == state)
    for(treatment in treatments){
      result <- att_gt(yname = party,
                    gname = treatment,
                    idname = 'AGS',
                    panel = TRUE,
                    tname = 'year',
                    xformla = ~ pop_density + unemployed + avg_age + female,
                    data = reg_data,
                    est_method = 'reg',
                    anticipation = 0,
                    control_group = 'nevertreated',
                    clustervars = c('AGS'),
                    bstrap = TRUE,
                    cband = TRUE,
                    allow_unbalanced_panel = TRUE,
                    
      )
    results[[length(results)+1]] <- result
    # calculate ATT
    att <- aggte(result, type = "group", bstrap = TRUE, clustervars = c('AGS'))
    # plot results
    result_fig <- ggdid(result, 
              ylim = c(floor(min(result$att - result$se * 2.345 - 1)), ceiling(max(result$att + result$se * 2.345 + 1))),
              ncol = 3, ax_text_size = 8, grtitle='', title_size = 10, legend=F) +
              labs(caption = sprintf('Overall ATT (SE): %.3f (%.3f) \n Wald-P: %.3f \n # Obs.: %s' ,
              att$overall.att, att$overall.se, result$Wpval, result$n)) +
              theme(plot.caption = element_text(hjust=0.5, size=8),
              panel.background = element_rect(fill='transparent'),
              plot.background = element_rect(fill='transparent', color=NA),
              legend.background = element_rect(fill='transparent'),
              legend.box.background = element_rect(fill='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
    # Plot ATT
    att_fig <- ggdid(att, legend=F, x_lab='', ax_text_size = 8, ylab='', title='ATT by year', title_size = 10) + 
    labs(caption = sprintf('2005 ATT(SE): %.3f (%.3f)\n2010 ATT(SE): %.3f (%.3f)\n2014 ATT(SE): %.3f (%.3f)', 
              att$att.egt[[1]], att$se.egt[[1]], att$att.egt[[2]], att$se.egt[[2]], att$att.egt[[3]], att$se.egt[[3]])) + 
              theme(plot.caption = element_text(hjust=0.5, size=8), 
              axis.title.x = element_blank(),
              axis.text.y = element_text(angle=90, size=6, hjust=0.5),
              panel.background = element_rect(fill='transparent'),
              plot.background = element_rect(fill='transparent', color=NA),
              legend.background = element_rect(fill='transparent'),
              legend.box.background = element_rect(fill='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
    # Combine and annotate Group and ATT plot
    combined_fig <- ggarrange(result_fig, att_fig, widths = c(3, 1)) + 
                    theme(plot.margin=unit(c(0,0,0,0),"cm"))
    combined_fig_anno <- annotate_figure(combined_fig,
                  top = text_grob(sprintf('%s', treatments[[treatment]])))
    figures[[length(figures)+1]] <- combined_fig_anno
    }
    # Combine Figures for all treatments
    arranged_fig <- ggarrange(plotlist=figures, nrow = 4, ncol = 1, common.legend = TRUE)
    # get legend
    result_fig <- ggdid(result) +
                theme(
                legend.background = element_rect(fill='transparent', color='transparent'),
                legend.box.background = element_rect(fill='transparent', color='transparent'),
                plot.margin=unit(c(0,0,0,0),"cm"))
    leg <- get_legend(result_fig)
    # add legend
    arranged_fig_leg <- ggarrange(arranged_fig, leg, nrow = 2, ncol = 1, heights = c(10, 1))
    final_fig <- annotate_figure(arranged_fig_leg,
                    top = text_grob(sprintf("LTW %s \nEffect on %s's vote share", states_dict[[state]], party), face = "bold", size = 14),
                    bottom = text_grob('Control Group: Never Treated. Estimation Method: Regression \n SE clustered on the municipality level.', size = 10),)
    ggsave(sprintf('%s_%s.png', state, party), plot = final_fig, path = sprintf('%s/figures/R/LTW/by_state_controls', path), units = 'cm', width = 21, height = 29.7, dpi="print")
    print(party)
  }
}